In [3]:
!pip install langchain openai pypdf PyMuPDF faiss-cpu python-dotenv ipywidgets


Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension



Defaulting to user installation because normal site-packages is not writeable


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [7]:
!pip install -U langchain-community langchain-core


Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install -U langchain


Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [13]:
!pip install -U langchain-openai


Defaulting to user installation because normal site-packages is not writeable


In [15]:
import langchain
print(langchain.__version__)

0.3.25


In [17]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings  # Corrected here
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI  # Corrected here
from langchain.chains import ConversationalRetrievalChain
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display

load_dotenv()


os.environ["OPENAI_API_KEY"] = "Your_API_key"


In [19]:
# Load and split the PDF content
loader = PyMuPDFLoader("Desktop/revision_exercise_sample.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Check loaded content
print(f"Loaded {len(texts)} chunks from the PDF.")


Loaded 14 chunks from the PDF.


In [21]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

# Save index locally for reuse
vectorstore.save_local("faiss_index")


In [23]:
# Setup conversational AI retrieval chain
llm = ChatOpenAI(temperature=0.1)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, 
    vectorstore.as_retriever()
)

chat_history = []


In [25]:
def ai_tutor(question):
    global chat_history
    result = qa_chain({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    return result["answer"]

# Simple widget interface
question_box = widgets.Text(
    value='',
    placeholder='Ask a question...',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

def on_submit(b):
    with output_area:
        output_area.clear_output(wait=True)
        answer = ai_tutor(question_box.value)
        print("AI Tutor:", answer)

submit_button = widgets.Button(description='Ask AI Tutor')
submit_button.on_click(on_submit)

display(question_box, submit_button, output_area)


Text(value='', description='Question:', placeholder='Ask a question...')

Button(description='Ask AI Tutor', style=ButtonStyle())

Output()